<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA, LSA, and BERTopic. The following information should be reported:

(1) Features (text representation) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster.


In [22]:
# Write your code here
import pandas as pd
imdb_data = pd.read_csv('/content/sentimentanalysis_reddit_data.csv')
print(imdb_data.shape)

(10000, 3)


In [23]:
import nltk   #cleaning
imdb_data['clean_text'] = imdb_data['clean_text'].str.replace('[^\w\s]','')  #removal of punctuation
imdb_data['clean_text'] = imdb_data['clean_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))#lower case
from nltk.corpus import stopwords #removal of stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
imdb_data['clean_text'] = imdb_data['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
words = []   #building document-term matrix
from nltk.tokenize import RegexpTokenizer
from gensim import corpora,models
tokenizer = RegexpTokenizer(r'\w+')
for x in pd.Series(imdb_data['clean_text']):
  a = tokenizer.tokenize(x)
  words.append(a)
dictionary = corpora.Dictionary(words)
corpus = [dictionary.doc2bow(word) for word in words]

In [26]:
import gensim  #LDA model
model = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word = dictionary,passes = 20)

In [27]:
print(model.print_topics(num_topics = 10, num_words = 5))

[(0, '0.065*"jurassic" + 0.049*"park" + 0.032*"watching" + 0.032*"data" + 0.032*"dr"'), (1, '0.042*"jurassic" + 0.030*"park" + 0.024*"s" + 0.012*"first" + 0.012*"work"'), (2, '0.066*"jurassic" + 0.059*"park" + 0.037*"t" + 0.037*"rex" + 0.029*"flare"'), (3, '0.067*"opening" + 0.048*"time" + 0.029*"film" + 0.029*"weekend" + 0.029*"1"'), (4, '0.098*"jurassic" + 0.068*"park" + 0.019*"like" + 0.019*"champagne" + 0.019*"glasses"'), (5, '0.139*"jurassic" + 0.070*"world" + 0.069*"park" + 0.025*"poster" + 0.019*"people"'), (6, '0.108*"jurassic" + 0.075*"park" + 0.016*"re" + 0.016*"spielberg" + 0.016*"version"'), (7, '0.063*"jurassic" + 0.036*"jeff" + 0.036*"goldblum" + 0.036*"park" + 0.018*"r"'), (8, '0.080*"jurassic" + 0.048*"park" + 0.032*"williams" + 0.016*"world" + 0.016*"scene"'), (9, '0.089*"jurassic" + 0.064*"park" + 0.025*"like" + 0.025*"scene" + 0.025*"s"')]


# **Question 2: Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. Note: Cross-validation (5-fold or 10-fold) should be conducted. Here is the reference of cross-validation: https://scikit-learn.org/stable/modules/cross_validation.html.

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.

In [ ]:
# Write your code here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

# Load the dataset
df = pd.read_csv('/content/annotated-book_reviews_sentiment_analysis-1.csv')

# Assuming you have a 'text' column for the text data and a 'label' column for sentiment labels
# Replace 'text' and 'label' with your actual column names
X = df['User Reviews']
y = df['sentiment']

# Split the data into training and testing sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Features: TF-IDF representation of the text data
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1

# Model 1: Naive Bayes
nb_model = make_pipeline(TfidfVectorizer(max_features=5000, stop_words='english'), MultinomialNB())
nb_scores = cross_val_score(nb_model, X, y, cv=5)  # 5-fold cross-validation

# Model 2: Logistic Regression
lr_model = make_pipeline(TfidfVectorizer(max_features=5000, stop_words='english'), LogisticRegression())
lr_scores = cross_val_score(lr_model, X, y, cv=5)  # 5-fold cross-validation

# Display results
print("Naive Bayes Cross-Validation Scores:", nb_scores)
print("Mean Accuracy: {:.2f}".format(nb_scores.mean()))

print("\nLogistic Regression Cross-Validation Scores:", lr_scores)
print("Mean Accuracy: {:.2f}".format(lr_scores.mean()))

# Train and evaluate models on the test set
nb_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

print("\nEvaluation on Test Set:")
print("\nNaive Bayes:")
nb_accuracy, nb_precision, nb_recall, nb_f1 = evaluate_model(nb_model, X_test, y_test)
print("Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(nb_accuracy, nb_precision, nb_recall, nb_f1))

print("\nLogistic Regression:")
lr_accuracy, lr_precision, lr_recall, lr_f1 = evaluate_model(lr_model, X_test, y_test)
print("Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(lr_accuracy, lr_precision, lr_recall, lr_f1))

# **Question 3: House price prediction**

(20 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download from canvas. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878.


In [2]:
df_test = pd.read_csv("/content/test.csv")
df_train = pd.read_csv("/content/train.csv")

In [3]:
print(df_test.shape)
print(df_train.shape)

(1459, 80)
(1460, 81)


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [6]:
df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

df_train = df_train.select_dtypes(include=['number']).interpolate().dropna()
df_test = df_test.select_dtypes(include=['number']).interpolate().dropna()

x_training_data = df_train.drop(['SalePrice','Id'], axis=1)
y_training_data = np.log(df_train.SalePrice)


x_train, x_test, y_train, y_test = train_test_split(x_training_data,y_training_data,random_state = 40, test_size=0.3)


regression = LinearRegression()
regression.fit(x_train,y_train)

y_pred = regression.predict(x_test)

In [9]:
print('Linear Regression of R squared is": %.4f' % regression.score(x_test, y_test))

Linear Regression of R squared is": 0.8804


In [12]:
import numpy as np
from sklearn.metrics import mean_squared_error
linreg = mean_squared_error(np.exp(y_pred), y_test)
lin_r = np.sqrt(linreg)
print(lin_r)

188802.18310280904


In [11]:
res = {"Predicted Prices":np.exp(y_pred),"Actual Prices":np.exp(y_test)}
df_val = pd.DataFrame(res)
df_val["Percentage Difference"] = round(abs((df_val["Predicted Prices"] - df_val["Actual Prices"]) / df_val["Actual Prices"]) * 100,2)
df_val

,Predicted Prices,Actual Prices,Percentage Difference
1258,187373.644622,190000.0,1.38
1041,178976.951042,173000.0,3.45
1181,251633.343662,392500.0,35.89
1108,180193.538586,181000.0,0.45
554,266967.773811,284000.0,6.00
...,...,...,...
1288,228544.745849,278000.0,17.79
1163,120901.108846,108959.0,10.96
1339,119370.516128,128500.0,7.10
1301,151233.379404,177500.0,14.80


# **Question 4: Using Pre-trained LLMs**

(20 points)
Utilize a **pre-trained Large Language Model (LLM) from the Hugging Face Repository** for your specific task using the data collected in Assignment 3. After creating an account on Hugging Face (https://huggingface.co/), choose a relevant LLM from their repository, such as GPT-3, BERT, or RoBERTa or any Meta based text analysis model. Provide a brief description of the selected LLM, including its original sources, significant parameters, and any task-specific fine-tuning if applied.

Perform a detailed analysis of the LLM's performance on your task, including key metrics, strengths, and limitations. Additionally, discuss any challenges encountered during the implementation and potential strategies for improvement. This will enable a comprehensive understanding of the chosen LLM's applicability and effectiveness for the given task.


In [25]:
pip install --upgrade numpy


In [26]:
pip show transformers


Name: transformers
Version: 4.35.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: sentence-transformers


In [28]:
!pip uninstall transformers


Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.35.2


In [29]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 33.9 MB/s eta 0:00:00


In [1]:
# Write your code here

import pandas as pd
from transformers import pipeline
df = pd.read_csv('sentimentanalysis_reddit_data.csv')
Check for any missing values
df = df.dropna(subset=['clean_text', 'sentiment'])
emotion_pipeline = pipeline("sentiment-analysis", model="distilroberta-base")
def map_sentiment_to_emotion(sentiment_label):


    if sentiment_label == 'positive':
        return 'joy'
    elif sentiment_label == 'negative':
        return 'sadness'
    else:
        return 'neutral'

df['emotion'] = df['sentiment'].apply(lambda x: map_sentiment_to_emotion(x.lower()))
df['predicted_emotion'] = df['clean_text'].apply(lambda x: emotion_pipeline(x)[0]['label'])

# Display the results
print(df[['clean_text', 'sentiment', 'emotion', 'predicted_emotion']])


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

                                             clean_text sentiment  emotion  \
0                           Jurassic Park Deleted Scene   neutral  neutral   
1                           Jurassic Park deleted scene   neutral  neutral   
2     So, I've removed some animations from Jurassic...   neutral  neutral   
3     During the filming of Jurassic Park (1993), T-...   neutral  neutral   
4     My daughter watching Jurassic Bark for the fir...   neutral  neutral   
...                                                 ...       ...      ...   
9995  People would probably still visit Jurassic Par...  positive      joy   
9996  In Jurassic Park, you can see Dr. Wu erasing a...  negative  sadness   
9997                       Jurassic Parks & Recreation.   neutral  neutral   
9998                   Shitty New Jurassic World Poster  negative  sadness   
9999          New Poster for 'Jurassic World: Dominion'   neutral  neutral   

     predicted_emotion  
0              LABEL_1  
1            

In [16]:
print("sentimental distribution:")
print(df['sentiment'].value_counts())
print("\n Predicted Emotion Label Distribution:")
print(df['predicted_emotion'].value_counts())

sentimental distribution:
sentiment
neutral     5500
positive    2600
negative    1900
Name: count, dtype: int64

 Predicted Emotion Label Distribution:
predicted_emotion
LABEL_1    10000
Name: count, dtype: int64
